In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from fairBalance import *
from util import *

In [2]:
def balancing(X_train, target, knn, sensitive_attribute, features, drop_features, continous_features):
    fcb = fairBalance(X_train, features, continous_features, drop_features, sensitive_attribute, target, knn = knn)
    fcb.fit()
    X_balanced, y_balanced = fcb.generater()
    
    return X_balanced, y_balanced

In [3]:
def prediction(X_train, y_train, X_test, features):
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test[features])

    return y_pred

In [4]:
def evaluation(X_test, y_pred, target, sensitive_attribute, priority_group):
    y_test = X_test[target]

    df_pred = pd.DataFrame()
    df_pred[sensitive_attribute]=X_test[sensitive_attribute].tolist()
    df_pred['truth']=y_test.tolist()
    df_pred['pred']=y_pred
    print("---- overall performance ----")
    performance(df_pred)
    print("---- performance of different groups ----")
    group_comp(df_pred, sensitive_attribute, priority_group)

## An example with COMPAS dataset

#### 1. reading the data file

In [10]:
df=pd.read_csv('../compas.csv')

#### 2. set all required parameters

In [11]:
knn=5
target='two_year_recid'
drop_features=['race','sex']
features=list(set(df.keys().tolist())-set(drop_features+[target]))
continous_features=['age','juv_fel_count','juv_misd_count','juv_other_count','priors_count']
sensitive_attribute = 'race'
privileged_group = 1

#### 3. separate train-test sets

In [12]:
X_train, X_test = train_test_split(df, test_size=0.2, stratify=df[target], random_state=0)

#### 4. balance the train set

In [13]:
X_balanced, y_balanced = balancing(X_train, target, knn, sensitive_attribute, features, drop_features, continous_features)

Cluster the original dataset into 6 clusters:
Removing 1233 samples from the original dataset...


#### 5.prediction & performance evaluation

In [14]:
print("========= Performance before balancing ======== ")
y_pred = prediction(X_train[features], X_train[target], X_test, features)
evaluation(X_test, y_pred, target, sensitive_attribute, priority_group = 1)

print("======== Performance after balancing ======== ")
y_pred_bal = prediction(X_balanced, y_balanced, X_test, features)
evaluation(X_test, y_pred_bal, target, sensitive_attribute, priority_group = 1)

========= Performance before balancing ======== 
---- overall performance ----
Test Accuracy  0.6766612641815235
Precision: 0.675828
Recall: 0.676661
F1: 0.673415
---- performance of different groups ----
+----------------+-------+-------+-------+-------+
| Group          |    F1 |   TPR |   FPR |    PR |
|----------------+-------+-------+-------+-------|
| Privileged     | 0.668 | 0.449 | 0.172 | 0.279 |
| Non-privileged | 0.673 | 0.624 | 0.277 | 0.447 |
+----------------+-------+-------+-------+-------+
Equal Opportunity 0.174
Equal Odds 0.140
Statistical Parity 0.167
======== Performance after balancing ======== 
---- overall performance ----
Test Accuracy  0.63290113452188
Precision: 0.631774
Recall: 0.632901
F1: 0.632078
---- performance of different groups ----
+----------------+-------+-------+-------+-------+
| Group          |    F1 |   TPR |   FPR |    PR |
|----------------+-------+-------+-------+-------|
| Privileged     | 0.657 | 0.525 | 0.256 | 0.360 |
| Non-privileged |